In [ ]:
# Data Source - https://drive.google.com/file/d/14bx0LTu1VKZ7nr9OEvWXIkc8J9nODjuO/view?usp=sharing

In [ ]:
# Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Install keras tuner everytime you restart colab
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.2 MB/s eta 0:00:00


In [ ]:
# Import Keras tuner
import keras_tuner
from kerastuner.tuners import RandomSearch

<ipython-input-4-3ac42f7db0c6>:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [ ]:
# importing the data
data = pd.read_csv('/content/drive/MyDrive/Datasets/Input/AQI.csv')
data.shape

(1093, 9)

In [ ]:
# data sample
data.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [ ]:
# Independent and dependent feature
x = data.iloc[:, :-1]     # independent
y = data.iloc[:, -1]      # dependent

In [ ]:
# train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Standardizing the data
scale = StandardScaler()
x_train_scaled = scale.fit_transform(x_train)
x_test_scaled = scale.transform(x_test)

In [ ]:
def build_model(hp):     # class

    # initialize the model
    model = Sequential()

    # --- FORWARD PROPAGATION --- #

    # Input layer
    model.add(Dense(units=8, kernel_initializer='he_uniform', input_dim=8))

    # hidden layer
    for i in range(hp.Int('num_layers', 2, 5)):
        model.add(Dense(units=hp.Int('units_'+str(i),
                                     min_value=2,
                                     max_value=6,
                                     step=2),
                        kernel_initializer='he_uniform',
                        activation=hp.Choice('activation', ['relu', 'linear', 'tanh'])))

    # output layer
    model.add(Dense(units=1, activation='relu'))

    # --- BACKWARD PROPAGATION --- #
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_squared_error',
        metrics=['accuracy'])

    return model

In [ ]:
# Spin the tuner
tuner = RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    directory="my_dir",
    project_name="AQI")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# tuner summary
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 2, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'linear', 'tanh'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 2, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
# fit the data onto the tuner
tuner.search(x_train_scaled, y_train, epochs=5, validation_data=(x_test_scaled, y_test))

Trial 3 Complete [00h 00m 10s]
val_accuracy: 0.009132419945672154

Best val_accuracy So Far: 0.013698630034923553
Total elapsed time: 00h 01m 09s


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/AQI
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
num_layers: 3
units_0: 4
activation: tanh
units_1: 2
learning_rate: 0.0001
units_2: 2
Score: 0.013698630034923553

Trial 1 summary
Hyperparameters:
num_layers: 5
units_0: 6
activation: relu
units_1: 2
learning_rate: 0.001
units_2: 6
units_3: 2
units_4: 2
Score: 0.013698630034923553

Trial 2 summary
Hyperparameters:
num_layers: 4
units_0: 4
activation: tanh
units_1: 2
learning_rate: 0.0001
units_2: 2
units_3: 6
units_4: 2
Score: 0.009132419945672154
